Автор материала: Зраев Артем.

Можно использовать в каких угодно целях.

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("./lection_materials/web1/WA_Fn-UseC_-Telco-Customer-Churn.csv")

##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [2]:
df_male = df[df['gender'] == 'Male']
df_female = df[df['gender'] == 'Female']

print(df['gender'].value_counts())
print(f'\nMale: {round(100 * (df_male.shape[0] / df.shape[0]), 2)}%')
print(f'Female: {round(100 * (df_female.shape[0] / df.shape[0]), 2)}%')

Male      3555
Female    3488
Name: gender, dtype: int64

Male: 50.48%
Female: 49.52%


##### 2. Какое количество уникальных значений у поля InternetService?

In [3]:
df['InternetService'].unique()

array(['DSL', 'Fiber optic', 'No'], dtype=object)

##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [4]:
df_total_charges = df['TotalCharges']

print(f'Median: {df_total_charges.median()}')
print(f'Mean: {df_total_charges.mean()}')
print(f'Std: {df_total_charges.std()}')

TypeError: 

В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

Тип данных object. Чтобы посчитать статистики, сперва необходимо преобразовать к числовому типу.

##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [5]:
df['PhoneService'] = df['PhoneService'].apply(lambda x: 1 if x == 'Yes' else 0)
df['PhoneService'].head()

0    0
1    1
2    1
3    0
4    1
Name: PhoneService, dtype: int64

##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [6]:
import re

df['TotalCharges'] = df['TotalCharges'].apply(lambda x: np.nan if len(re.findall('\s+',x)) > 0 else x)
df['TotalCharges'] = df['TotalCharges'].astype('float32')
df['TotalCharges'].fillna(0, axis=0, inplace=True)

df_total_charges = df['TotalCharges']
print(f'Median: {df_total_charges.median()}')
print(f'Mean: {df_total_charges.mean()}')
print(f'Std: {df_total_charges.std()}')

Median: 1394.550048828125
Mean: 2279.734375
Std: 2266.79443359375


##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [7]:
df['Churn'] = df['Churn'].apply(lambda x: 1 if x == 'Yes' else 0)
df['Churn'].head()

0    0
1    0
2    1
3    0
4    1
Name: Churn, dtype: int64

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [8]:
df[['StreamingMovies', 'StreamingTV', 'TechSupport']].head()

,StreamingMovies,StreamingTV,TechSupport
0,No,No,No
1,No,No,No
2,No,No,No
3,No,No,Yes
4,No,No,No


In [9]:
def feature_transform(x):
    if x == 'Yes':
        return 1
    elif x == 'No' or x == 'No internet service':
        return 0
    else:
        return 0

features = ['StreamingMovies', 'StreamingTV', 'TechSupport']
for feature in features:
    df[feature] = df[feature].apply(lambda x: feature_transform(x))
df[features].head()

,StreamingMovies,StreamingTV,TechSupport
0,0,0,0
1,0,0,0
2,0,0,0
3,0,0,1
4,0,0,0


##### 8. Заполните пропуски в поле PhoneService значением 0

In [10]:
df['PhoneService'].fillna(0, axis=0, inplace=True)

#### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [11]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']

df = df[columns]
df.head(3)

,gender,tenure,PhoneService,TotalCharges,StreamingMovies,StreamingTV,TechSupport,Churn
0,Female,1,0,29.850000,0,0,0,0
1,Male,34,1,1889.500000,0,0,0,0
2,Male,2,1,108.150002,0,0,0,1


##### 9. Разделите датасет на тренировочную и тестовую выборку (подсказка - воспользуйтесь train_test_split из sklearn.model_selection. Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)

In [12]:
from sklearn.model_selection import train_test_split

features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'

X = df[features]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.3,
    random_state=0,
)

##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [13]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline([
                ('selector', ColumnSelector(key='gender')),
                ('ohe', OHEEncoder(key='gender'))
            ])

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [14]:
gender.fit_transform(X_train, y_train)


,gender_Female,gender_Male
3580,1,0
2364,0,1
6813,1,0
789,0,1
561,0,1
...,...,...
4931,0,1
3264,1,0
1653,0,1
2607,0,1


##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

In [15]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
    ('selector', NumberSelector(key='tenure')),
    ('standard', StandardScaler())
])

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [16]:
total_charges = Pipeline([
    ('selector', NumberSelector(key='TotalCharges')),
    ('standard', StandardScaler())
])

Объединение всех "кубиков" очень легко сделать таким образом

In [17]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
    ('selector', ColumnSelector(key=['PhoneService', 'StreamingMovies', 'StreamingTV', 'TechSupport']))
])

In [18]:
feats = FeatureUnion([
    ('tenure', tenure),
    ('TotalCharges', total_charges),
    ('continuos_features', number_features),
    ('gender', gender)
])

feature_processing = Pipeline([
    ('feats', feats)
])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [19]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

pipeline.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('tenure',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='tenure')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('TotalCharges',
                                                 Pipeline(steps=[('selector',
                                                                  NumberSelector(key='TotalCharges')),
                                                                 ('standard',
                                                                  StandardScaler())])),
                                                ('continuos_features',
                                                 Pipeline(steps=[('selector',
          

##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [20]:
y_pred = pipeline.predict(X_test)
set(y_pred)

{0, 1}

##### 15. Посчитайте метрики качества получившейся модели (roc_auc, logloss)

In [21]:
from sklearn.metrics import roc_auc_score, log_loss

print(f'roc_auc: {roc_auc_score(y_test, pipeline.predict_proba(X_test)[:, 1])}')
print(f'logloss: {log_loss(y_test, y_pred)}')


roc_auc: 0.7843673204432697
logloss: 8.565307119153056


### Сохраним наш пайплайн

In [22]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)